## makemore: becoming a backprop ninja

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [3]:
# ----------------- Vocabulary Building ----------------------

# Build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)

# ----------------- Output Vocabulary ----------------------

print(itos)
print(vocab_size)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# ----------------- Dataset Building Function ----------------------

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]  # Crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

# ----------------- Setup ----------------------

block_size = 3  # Context length: how many characters do we take to predict the next one?

# ----------------- Data Preparation ----------------------

random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

# ----------------- Dataset Creation ----------------------

Xtr, Ytr = build_dataset(words[:n1])  # 80% for training
Xdev, Ydev = build_dataset(words[n1:n2])  # 10% for validation
Xte, Yte = build_dataset(words[n2:])  # 10% for testing


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
# Constants
n_embd = 10  # Dimensionality of the character embedding vectors
n_hidden = 64  # Number of neurons in the hidden layer of the MLP
vocab_size = 100  # Example value, replace with actual vocabulary size
block_size = 5  # Example value, replace with actual block size

# Random generator for reproducibility
g = torch.Generator().manual_seed(2147483647)

# Initializing parameters
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5 / 3) / (n_embd * block_size)**0.5
b1 = torch.randn(n_hidden, generator=g) * 0.1  # Using b1 just for fun, it's useless because of BN
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# BatchNorm parameters
bngain = torch.randn((1, n_hidden), generator=g) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden), generator=g) * 0.1

# List of parameters
parameters = [C, W1, b1, W2, b2, bngain, bnbias]

# Print total number of parameters
print(sum(p.nelement() for p in parameters))

# Enable gradient computation for all parameters
for p in parameters:
    p.requires_grad = True


10892


In [7]:
# ----------------- Minibatch Construction ----------------------

# Parameters
batch_size = 32
n = batch_size  # A shorter variable also, for convenience

# Construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]  # Batch X, Y

In [8]:
# Forward pass, "chunkated" into smaller steps that are possible to backward one at a time

# Embedding layer
emb = C[Xb]  # Embed the characters into vectors
embcat = emb.view(emb.shape[0], -1)  # Concatenate the vectors

# ----------------- Layer 1 ----------------------

# Linear layer 1
hprebn = embcat @ W1 + b1  # Hidden layer pre-activation

# BatchNorm layer
bnmeani = hprebn.mean(0, keepdim=True)
bndiff = hprebn - bnmeani
bnvar = bndiff.var(0, unbiased=True, keepdim=True)  # Note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5).pow(-0.5)
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact)  # Hidden layer

# ----------------- Layer 2 ----------------------

# Linear layer 2
logits = h @ W2 + b2  # Output layer

# ----------------- Cross Entropy Loss ----------------------

# Cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes  # Subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum.pow(-1)  # Using counts_sum.pow(-1) for backprop to be bit exact
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# ----------------- Backward Pass ----------------------

# PyTorch backward pass
for p in parameters:
    p.grad = None

# Retain gradients for intermediate variables
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes, logits, 
          h, hpreact, bnraw, bnvar_inv, bnvar, bndiff, hprebn, bnmeani, embcat, emb]:
    t.retain_grad()

# Backward pass
loss.backward()

loss


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x30 and 50x64)

In [ ]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

# cmp('logprobs', dlogprobs, logprobs)
# cmp('probs', dprobs, probs)
# cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# cmp('counts_sum', dcounts_sum, counts_sum)
# cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h', dh, h)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpreact', dhpreact, hpreact)
# cmp('bngain', dbngain, bngain)
# cmp('bnbias', dbnbias, bnbias)
# cmp('bnraw', dbnraw, bnraw)
# cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
# cmp('bnvar', dbnvar, bnvar)
# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

In [ ]:
# backward pass

# -----------------
# YOUR CODE HERE :)
dlogits = None # TODO. my solution is 3 lines
# -----------------

#cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

In [ ]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

In [ ]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = None # TODO. my solution is 1 (long) line
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

In [ ]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# ----------------- Initialization ----------------------

# Initialize parameters
n_embd = 10  # The dimensionality of the character embedding vectors
n_hidden = 200  # The number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)  # For reproducibility
C = torch.randn((vocab_size, n_embd), generator=g)

# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1

# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))  # Number of parameters in total
for p in parameters:
    p.requires_grad = True

# ----------------- Optimization Setup ----------------------

max_steps = 200000
batch_size = 32
n = batch_size  # Convenience
lossi = []

# ----------------- Training Loop ----------------------

# Kick off optimization
for i in range(max_steps):

    # Minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]  # Batch X, Y

    # ----------------- Forward Pass ----------------------

    # Embedding layer
    emb = C[Xb]  # Embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1)  # Concatenate the vectors

    # Linear layer
    hprebn = embcat @ W1 + b1  # Hidden layer pre-activation

    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5).pow(-0.5)
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------

    # Non-linearity
    h = torch.tanh(hpreact)  # Hidden layer

    # Output layer
    logits = h @ W2 + b2

    # Loss function
    loss = F.cross_entropy(logits, Yb)

    # ----------------- Backward Pass ----------------------

    for p in parameters:
        p.grad = None

    loss.backward()  # Use this for correctness comparisons, delete it later!

    # Manual backpropagation
    # -----------------
    # YOUR CODE HERE :)
    dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # ----------------- Parameter Update ----------------------

    lr = 0.1 if i < 100000 else 0.01  # Step learning rate decay
    for p, grad in zip(parameters, grads):
        p.data += -lr * p.grad  # Old way of cheems doge (using PyTorch grad from .backward())
        # p.data += -lr * grad # New way of swole doge TODO: enable

    # ----------------- Tracking and Logging ----------------------

    # Track stats
    if i % 10000 == 0:  # Print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    if i >= 100:  # TODO: delete early breaking when you're ready to train the full net
        break


In [ ]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [ ]:
# ----------------- Evaluation Function ----------------------

@torch.no_grad()  # This decorator disables gradient tracking
def split_loss(split):
    # ----------------- Data Selection ----------------------
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    
    # ----------------- Forward Pass ----------------------
    emb = C[x]  # (N, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1)  # Concatenate into (N, block_size * n_embd)
    hpreact = embcat @ W1 + b1
    
    # ----------------- BatchNorm Layer ----------------------
    hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5).pow(-0.5) + bnbias
    
    # ----------------- Non-linearity ----------------------
    h = torch.tanh(hpreact)  # (N, n_hidden)
    
    # ----------------- Output Layer ----------------------
    logits = h @ W2 + b2  # (N, vocab_size)
    
    # ----------------- Loss Calculation ----------------------
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

# ----------------- Evaluation ----------------------
split_loss('train')
split_loss('val')


In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [ ]:
# ----------------- Sampling from the Model ----------------------
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size  # Initialize with all zeros

    while True:
        # ----------------- Forward Pass ----------------------
        emb = C[torch.tensor([context])]  # (1, block_size, n_embd)
        embcat = emb.view(emb.shape[0], -1)  # Concatenate into (N, block_size * n_embd)
        hpreact = embcat @ W1 + b1

        # ----------------- BatchNorm Layer ----------------------
        hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5).pow(-0.5) + bnbias

        # ----------------- Non-linearity ----------------------
        h = torch.tanh(hpreact)  # (N, n_hidden)

        # ----------------- Output Layer ----------------------
        logits = h @ W2 + b2  # (N, vocab_size)

        # ----------------- Sampling ----------------------
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)

        # ----------------- End Condition ----------------------
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))
